In [1]:
import seaborn
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt

import plotly.graph_objects as go

In [2]:
! pip install plotly

In [81]:
with open("../results/metric_results.json", "r") as file:
    results_random_search = json.load(file) 

In [82]:
results_random_search.keys()

dict_keys(['block_size', 'search_area', 'time', 'msne', 'pepn', 'estimators'])

In [83]:
print(len(results_random_search["estimators"]))

25


In [84]:
padding = ["mse" for i in range(len(results_random_search["block_size"]) - len(results_random_search["estimators"]))]

In [85]:
results_random_search["estimators"] = padding + results_random_search["estimators"]

In [86]:
df = pd.DataFrame(results_random_search)

In [87]:
df.head()

,block_size,search_area,time,msne,pepn,estimators
0,30,15,2.941297,9.928420,0.830519,mse
1,90,55,17.619460,15.194300,0.990635,mse
2,70,40,11.234741,12.930214,0.972721,mse
3,85,50,15.876823,14.558275,0.973756,mse
4,45,60,32.179989,14.592048,0.979038,mse


In [10]:
df

,block_size,search_area,time,msne,pepn,estimators
0,30,15,2.941297,9.928420,0.830519,mse
1,90,55,17.619460,15.194300,0.990635,mse
2,70,40,11.234741,12.930214,0.972721,mse
3,85,50,15.876823,14.558275,0.973756,mse
4,45,60,32.179989,14.592048,0.979038,mse
...,...,...,...,...,...,...
96,55,20,0.325773,5.229914,0.362676,ccorr
97,50,28,0.320457,3.877706,0.277820,ccorr
98,35,38,0.464758,2.918335,0.187214,ccorr
99,40,30,0.403068,3.651379,0.222678,ccorr


In [67]:
import plotly.express as px
df_copy = df.loc[(df["time"] < 120) & (df["msne"]<20)]
df_copy["msne"] = (1 / df["msne"]) * 200
fig = px.scatter_3d(df_copy, x='block_size', y='search_area', z='time',
              color='estimators', size='msne', size_max=30,
              symbol='estimators', opacity=1, width=1000, height=1000, template= "seaborn")

# tight layout
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(showgrid=True, gridwidth=2, gridcolor='black', tickfont=dict(size=15), title_font=dict(size=20)),
        yaxis=dict(showgrid=True, gridwidth=2, gridcolor='black', tickfont=dict(size=15), title_font=dict(size=20)),
        zaxis=dict(showgrid=True, gridwidth=2, gridcolor='black', tickfont=dict(size=15), title_font=dict(size=20))
    )
)
fig.show()

/tmp/ipykernel_536683/588684547.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
df_copy = df.loc[(df["time"] < 120) & (df["msne"]<20)]

estimators_means = df_copy.groupby("estimators", as_index=False).mean(numeric_only=True)


fig = go.Figure(
    layout=dict(
        xaxis=dict(categoryorder="category descending"),
        yaxis=dict(range=[0, 20]),
        scattermode="group",
        legend=dict(groupclick="toggleitem"),
        width=700, height=700
    )
)



fig.add_trace(
    go.Bar(
        x=estimators_means.estimators,
        y=estimators_means.msne,
        name="Average",
        marker_color="IndianRed",
        offsetgroup="MSNE",
        legendgroup="MSNE",
        legendgrouptitle_text="MSNE",
    )
)

fig.add_trace(
    go.Bar(
        x=estimators_means.estimators,
        y=estimators_means.pepn*10,
        name="Average * 10",
        marker_color="green",
        offsetgroup="pepn",
        legendgroup="pepn",
        legendgrouptitle_text="pepn",
    )
)

fig.update_traces(width=0.1)

fig.add_trace(
    go.Scatter(
        x=df_copy.estimators,
        y=df_copy.msne,
        mode="markers",
        name="Individual msne",
        marker=dict(color="teal", size=5),
        offsetgroup="MSNE",
        legendgroup="MSNE",
    )
)




fig.add_trace(
    go.Scatter(
        x=df_copy.estimators,
        y=df_copy.pepn*10,
        mode="markers",
        name="Individual pepn * 10",
        marker=dict(color="red", size=3),
        offsetgroup="pepn",
        legendgroup="pepn",
    )
)

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(showgrid=True, gridwidth=2, gridcolor='black', tickfont=dict(size=15), title_font=dict(size=20)),
        yaxis=dict(showgrid=True, gridwidth=2, gridcolor='black', tickfont=dict(size=15), title_font=dict(size=20)),
        zaxis=dict(showgrid=True, gridwidth=2, gridcolor='black', tickfont=dict(size=15), title_font=dict(size=20))
    )
)

fig.show()
